In [4]:
import numpy as np
import warnings
import pickle
import os
from urllib.request import urlopen
import random as rand
%matplotlib inline
import matplotlib.pyplot as plt
import time

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, precision_recall_curve
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, multilabel_confusion_matrix, classification_report
from sklearn.metrics import label_ranking_average_precision_score, coverage_error, label_ranking_loss

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, SGDClassifier

from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, cross_val_predict, cross_val_score

from sklearn import svm as svm, neighbors as nbrs, tree as tree, gaussian_process as GPC, naive_bayes as NB

from sklearn.experimental import enable_hist_gradient_boosting

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, VotingClassifier

from sklearn.neural_network import MLPClassifier

dir = 'C:/ ... /'   # insert path here          
os.chdir(dir)

url_main = 'file:///C:/ ... /'
data_file_name = "3_states.pkl"
label_file_name = "3_labels.npy"

#DATA
data = pickle.load(urlopen(url_main + data_file_name))
data = np.unpackbits(data).reshape(-1, 2500)
data = data.astype('int')
#data[np.where(data==0)] = -1

#LABELS
labels = np.load(urlopen(url_main + label_file_name))
#labels = np.unpackbits(labels).reshape(30000, -1)

X = data
Y = labels

print('X shape:', X.shape)
print('Y shape:', Y.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, test_size = 0.2)

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)

#sorted(sklearn.metrics.SCORERS.keys()) UNCOMMENT TO PRINT ALL SCORERS

X shape: (30000, 2500)
Y shape: (30000, 1)
X_train shape: (24000, 2500)
Y_train shape: (24000, 1)


In [5]:
clf1 = LogisticRegression(solver='lbfgs', multi_class='multinomial', random_state=42, verbose=1, max_iter=1000)

clf2 = svm.LinearSVC(penalty='l1', loss='squared_hinge', dual=False, tol=0.0001, C=1.0, 
                    multi_class='ovr', fit_intercept=True, intercept_scaling=1, class_weight=None, 
                    verbose=1, random_state=42, max_iter=1000)

clf3 = SGDClassifier(alpha=0.01, shuffle=True, early_stopping=True, 
                    learning_rate='adaptive', loss='hinge', eta0=0.001,
                    penalty='l2', random_state=42)

clf4 = NB.MultinomialNB(alpha=1.5, fit_prior=True)

clf5 = tree.DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=42,
                                  min_samples_split=50, min_samples_leaf=1)

clf6 = RandomForestClassifier(oob_score=True, n_estimators=400, n_jobs=-1,random_state=42)

clf7 = ExtraTreesClassifier(oob_score=True, n_estimators=400, n_jobs=-1, bootstrap=True,
                            random_state=42)

clf8 = BaggingClassifier(RandomForestClassifier(n_estimators=400),
                         max_samples=0.5, max_features=0.5, n_jobs=-1,random_state=42)

clf95 = GradientBoostingClassifier(random_state=42)

clf9 = HistGradientBoostingClassifier(random_state=42)


clf10 = MLPClassifier(solver='lbfgs', hidden_layer_sizes = (5, 1000), alpha = 1e-08,
                      warm_start=True, verbose=1, random_state=42)


def benchmark(clf, CVV_n):
    
    print("\n", clf, "\n")

    y_train_pred = cross_val_predict(clf, X_train, Y_train.ravel(), n_jobs=-1)

    print(classification_report(Y_train, y_train_pred), '\n',
          confusion_matrix(Y_train, y_train_pred), '\n')

    t1=time.time()

    CVV=cross_validate(clf, X_train, Y_train.ravel(), cv=CVV_n, return_train_score=True, n_jobs=-1,
                       scoring = ['accuracy',
                                  'precision_micro', 'precision_macro', 'precision_weighted',
                                  'recall_micro', 'recall_macro', 'recall_weighted',
                                  'f1_macro', 'f1_weighted'
                                  ])
                                  
    t2=time.time()

    for cvv_keys in sorted(CVV.keys()):
        print(cvv_keys, "{0:.4f}".format(CVV[str(cvv_keys)].mean()),
              "{0:.4f}".format(CVV[str(cvv_keys)].std()))

    print()
    print("{0:.2f}".format(t2-t1)+'s')
    print()

In [6]:
benchmark(clf4, 20) # test benchmark, fastest classifier


 MultinomialNB(alpha=1.5, class_prior=None, fit_prior=True) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       0.43      0.48      0.46      7967
           1       0.37      0.31      0.34      8049
           2       0.45      0.47      0.46      7984

    accuracy                           0.42     24000
   macro avg       0.42      0.42      0.42     24000
weighted avg       0.42      0.42      0.42     24000
 
 [[3843 2222 1902]
 [2877 2513 2659]
 [2191 2068 3725]] 

fit_time 3.6847 1.1416
score_time 0.3851 0.1471
test_accuracy 0.4263 0.0102
test_f1_macro 0.4237 0.0104
test_f1_weighted 0.4235 0.0105
test_precision_macro 0.4237 0.0106
test_precision_micro 0.4263 0.0102
test_precision_weighted 0.4235 0.0106
test_recall_macro 0.4266 0.0102
test_recall_micro 0.4263 0.0102
test_recall_weighted 0.4263 0.0102
train_accuracy 0.4885 0.0011
train_f1_macro 0.4884 0.0011
train_f1_weighted 0.4883 0.0011
train_precision_macro 0.4884 0.0011
train_precision_micro 0.4885 0.0011
train_precision_weighted 0.4884 0.0011
train_recall_macr

In [148]:
benchmark(clf1, 20)


 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       0.35      0.38      0.37      7985
           1       0.37      0.33      0.35      8009
           2       0.35      0.37      0.36      8006

    accuracy                           0.36     24000
   macro avg       0.36      0.36      0.36     24000
weighted avg       0.36      0.36      0.36     24000
 
 [[3054 2142 2789]
 [2750 2618 2641]
 [2818 2229 2959]] 

fit_time 541.2055 93.1841
score_time 0.4731 0.0841
test_accuracy 0.3623 0.0121
test_f1_macro 0.3617 0.0121
test_f1_weighted 0.3617 0.0121
test_precision_macro 0.3624 0.0121
test_precision_micro 0.3623 0.0121
test_precision_weighted 0.3624 0.0121
test_recall_macro 0.3623 0.0121
test_recall_micro 0.3623 0.0121
test_recall_weighted 0.3623 0.0121
train_accuracy 0.5350 0.0016
train_f1_macro 0.5350 0.0017
train_f1_weighted 0.5350 0.0017
train_precision_macro 0.5350 0.0017
train_precision_micro 0.5350 0.0016
train_precision_weighted 0.5350 0.0017
train_recall_m

In [149]:
benchmark(clf2, 20)


 LinearSVC(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l1', random_state=42, tol=0.0001,
          verbose=1) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       0.35      0.37      0.36      7985
           1       0.37      0.33      0.35      8009
           2       0.36      0.37      0.37      8006

    accuracy                           0.36     24000
   macro avg       0.36      0.36      0.36     24000
weighted avg       0.36      0.36      0.36     24000
 
 [[2994 2276 2715]
 [2742 2662 2605]
 [2722 2303 2981]] 

fit_time 177.4388 20.3489
score_time 0.3067 0.1641
test_accuracy 0.3608 0.0126
test_f1_macro 0.3604 0.0127
test_f1_weighted 0.3604 0.0127
test_precision_macro 0.3607 0.0128
test_precision_micro 0.3608 0.0126
test_precision_weighted 0.3607 0.0128
test_recall_macro 0.3608 0.0126
test_recall_micro 0.3608 0.0126
test_recall_weighted 0.3608 0.0126
train_accuracy 0.5325 0.0020
train_f1_macro 0.5323 0.0020
train_f1_weighted 0.5324 0.0020
train_precision_macro 0.5324 0.0020
train_precision_micro 0.5325 0.0020
train_precision_weighted 0.5324 0.0020
train_recall_m

In [150]:
benchmark(clf3, 20)


 SGDClassifier(alpha=0.01, average=False, class_weight=None, early_stopping=True,
              epsilon=0.1, eta0=0.001, fit_intercept=True, l1_ratio=0.15,
              learning_rate='adaptive', loss='hinge', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=42, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       0.40      0.40      0.40      7985
           1       0.37      0.37      0.37      8009
           2       0.40      0.40      0.40      8006

    accuracy                           0.39     24000
   macro avg       0.39      0.39      0.39     24000
weighted avg       0.39      0.39      0.39     24000
 
 [[3207 2474 2304]
 [2472 2971 2566]
 [2277 2525 3204]] 

fit_time 41.4105 6.3357
score_time 0.3840 0.0979
test_accuracy 0.4036 0.0122
test_f1_macro 0.4037 0.0122
test_f1_weighted 0.4037 0.0122
test_precision_macro 0.4043 0.0124
test_precision_micro 0.4036 0.0122
test_precision_weighted 0.4043 0.0124
test_recall_macro 0.4036 0.0122
test_recall_micro 0.4036 0.0122
test_recall_weighted 0.4036 0.0122
train_accuracy 0.5381 0.0017
train_f1_macro 0.5370 0.0017
train_f1_weighted 0.5370 0.0017
train_precision_macro 0.5391 0.0016
train_precision_micro 0.5381 0.0017
train_precision_weighted 0.5390 0.0016
train_recall_mac

In [151]:
benchmark(clf4, 20)


 MultinomialNB(alpha=1.5, class_prior=None, fit_prior=True) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       0.45      0.48      0.46      7985
           1       0.36      0.32      0.34      8009
           2       0.46      0.47      0.46      8006

    accuracy                           0.42     24000
   macro avg       0.42      0.42      0.42     24000
weighted avg       0.42      0.42      0.42     24000
 
 [[3829 2312 1844]
 [2785 2594 2630]
 [1986 2263 3757]] 

fit_time 1.8796 0.7427
score_time 0.3019 0.0890
test_accuracy 0.4251 0.0142
test_f1_macro 0.4231 0.0140
test_f1_weighted 0.4231 0.0140
test_precision_macro 0.4225 0.0141
test_precision_micro 0.4251 0.0142
test_precision_weighted 0.4225 0.0141
test_recall_macro 0.4252 0.0142
test_recall_micro 0.4251 0.0142
test_recall_weighted 0.4251 0.0142
train_accuracy 0.4876 0.0019
train_f1_macro 0.4875 0.0019
train_f1_weighted 0.4875 0.0019
train_precision_macro 0.4874 0.0019
train_precision_micro 0.4876 0.0019
train_precision_weighted 0.4874 0.0019
train_recall_macr

In [152]:
benchmark(clf5, 20)


 DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=50,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best') 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       0.94      0.89      0.91      7985
           1       0.78      0.87      0.82      8009
           2       0.92      0.87      0.89      8006

    accuracy                           0.87     24000
   macro avg       0.88      0.87      0.88     24000
weighted avg       0.88      0.87      0.88     24000
 
 [[7077  898   10]
 [ 467 6944  598]
 [   8 1036 6962]] 

fit_time 21.9300 3.7161
score_time 0.1583 0.0334
test_accuracy 0.8827 0.0089
test_f1_macro 0.8840 0.0088
test_f1_weighted 0.8840 0.0088
test_precision_macro 0.8880 0.0087
test_precision_micro 0.8827 0.0089
test_precision_weighted 0.8879 0.0087
test_recall_macro 0.8827 0.0089
test_recall_micro 0.8827 0.0089
test_recall_weighted 0.8827 0.0089
train_accuracy 0.9139 0.0014
train_f1_macro 0.9150 0.0014
train_f1_weighted 0.9150 0.0014
train_precision_macro 0.9189 0.0015
train_precision_micro 0.9139 0.0014
train_precision_weighted 0.9188 0.0015
train_recall_mac

In [153]:
benchmark(clf6, 20)


 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7985
           1       1.00      1.00      1.00      8009
           2       1.00      1.00      1.00      8006

    accuracy                           1.00     24000
   macro avg       1.00      1.00      1.00     24000
weighted avg       1.00      1.00      1.00     24000
 
 [[7985    0    0]
 [   0 8009    0]
 [   0    0 8006]] 

fit_time 165.8159 29.1713
score_time 9.5042 5.4006
test_accuracy 1.0000 0.0000
test_f1_macro 1.0000 0.0000
test_f1_weighted 1.0000 0.0000
test_precision_macro 1.0000 0.0000
test_precision_micro 1.0000 0.0000
test_precision_weighted 1.0000 0.0000
test_recall_macro 1.0000 0.0000
test_recall_micro 1.0000 0.0000
test_recall_weighted 1.0000 0.0000
train_accuracy 1.0000 0.0000
train_f1_macro 1.0000 0.0000
train_f1_weighted 1.0000 0.0000
train_precision_macro 1.0000 0.0000
train_precision_micro 1.0000 0.0000
train_precision_weighted 1.0000 0.0000
train_recall_m

In [154]:
benchmark(clf7, 20)


 ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
                     oob_score=True, random_state=42, verbose=0,
                     warm_start=False) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7985
           1       1.00      1.00      1.00      8009
           2       1.00      1.00      1.00      8006

    accuracy                           1.00     24000
   macro avg       1.00      1.00      1.00     24000
weighted avg       1.00      1.00      1.00     24000
 
 [[7985    0    0]
 [   0 8009    0]
 [   0    0 8006]] 

fit_time 177.7945 30.0924
score_time 7.9448 3.6700
test_accuracy 1.0000 0.0000
test_f1_macro 1.0000 0.0000
test_f1_weighted 1.0000 0.0000
test_precision_macro 1.0000 0.0000
test_precision_micro 1.0000 0.0000
test_precision_weighted 1.0000 0.0000
test_recall_macro 1.0000 0.0000
test_recall_micro 1.0000 0.0000
test_recall_weighted 1.0000 0.0000
train_accuracy 1.0000 0.0000
train_f1_macro 1.0000 0.0000
train_f1_weighted 1.0000 0.0000
train_precision_macro 1.0000 0.0000
train_precision_micro 1.0000 0.0000
train_precision_weighted 1.0000 0.0000
train_recall_m

In [155]:
benchmark(clf8, 20)


 BaggingClassifier(base_estimator=RandomForestClassifier(bootstrap=True,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features='auto',
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        n_estimators=400,
                                                        n_jobs=None,
     

c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7985
           1       1.00      1.00      1.00      8009
           2       1.00      1.00      1.00      8006

    accuracy                           1.00     24000
   macro avg       1.00      1.00      1.00     24000
weighted avg       1.00      1.00      1.00     24000
 
 [[7985    0    0]
 [   0 8009    0]
 [   0    0 8006]] 

fit_time 376.2670 99.3751
score_time 48.6799 31.6970
test_accuracy 1.0000 0.0000
test_f1_macro 1.0000 0.0000
test_f1_weighted 1.0000 0.0000
test_precision_macro 1.0000 0.0000
test_precision_micro 1.0000 0.0000
test_precision_weighted 1.0000 0.0000
test_recall_macro 1.0000 0.0000
test_recall_micro 1.0000 0.0000
test_recall_weighted 1.0000 0.0000
train_accuracy 1.0000 0.0000
train_f1_macro 1.0000 0.0000
train_f1_weighted 1.0000 0.0000
train_precision_macro 1.0000 0.0000
train_precision_micro 1.0000 0.0000
train_precision_weighted 1.0000 0.0000
train_recall

In [156]:
benchmark(clf9, 3)


 HistGradientBoostingClassifier(l2_regularization=0.0, learning_rate=0.1,
                               loss='auto', max_bins=256, max_depth=None,
                               max_iter=100, max_leaf_nodes=31,
                               min_samples_leaf=20, n_iter_no_change=None,
                               random_state=42, scoring=None, tol=1e-07,
                               validation_fraction=0.1, verbose=0) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7985
           1       1.00      1.00      1.00      8009
           2       1.00      1.00      1.00      8006

    accuracy                           1.00     24000
   macro avg       1.00      1.00      1.00     24000
weighted avg       1.00      1.00      1.00     24000
 
 [[7985    0    0]
 [   0 8009    0]
 [   0    0 8006]] 

fit_time 288.2737 1.2362
score_time 4.7798 0.1173
test_accuracy 1.0000 0.0000
test_f1_macro 1.0000 0.0000
test_f1_weighted 1.0000 0.0000
test_precision_macro 1.0000 0.0000
test_precision_micro 1.0000 0.0000
test_precision_weighted 1.0000 0.0000
test_recall_macro 1.0000 0.0000
test_recall_micro 1.0000 0.0000
test_recall_weighted 1.0000 0.0000
train_accuracy 1.0000 0.0000
train_f1_macro 1.0000 0.0000
train_f1_weighted 1.0000 0.0000
train_precision_macro 1.0000 0.0000
train_precision_micro 1.0000 0.0000
train_precision_weighted 1.0000 0.0000
train_recall_ma

In [157]:
benchmark(clf10, 3)


 MLPClassifier(activation='relu', alpha=1e-08, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 1000), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=42, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=1, warm_start=True) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       0.76      0.88      0.81      7985
           1       0.80      0.66      0.72      8009
           2       0.94      0.94      0.94      8006

    accuracy                           0.83     24000
   macro avg       0.83      0.83      0.83     24000
weighted avg       0.83      0.83      0.83     24000
 
 [[7042  943    0]
 [2208 5282  519]
 [  61  391 7554]] 

fit_time 205.3077 3.1407
score_time 1.9140 0.1152
test_accuracy 0.8282 0.0045
test_f1_macro 0.8254 0.0055
test_f1_weighted 0.8253 0.0055
test_precision_macro 0.8302 0.0044
test_precision_micro 0.8282 0.0045
test_precision_weighted 0.8303 0.0044
test_recall_macro 0.8283 0.0045
test_recall_micro 0.8282 0.0045
test_recall_weighted 0.8282 0.0045
train_accuracy 0.9807 0.0026
train_f1_macro 0.9807 0.0026
train_f1_weighted 0.9807 0.0026
train_precision_macro 0.9810 0.0026
train_precision_micro 0.9807 0.0026
train_precision_weighted 0.9810 0.0026
train_recall_ma

In [159]:
benchmark(clf95, 3)


 GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False) 



c:\python373\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7985
           1       1.00      1.00      1.00      8009
           2       1.00      1.00      1.00      8006

    accuracy                           1.00     24000
   macro avg       1.00      1.00      1.00     24000
weighted avg       1.00      1.00      1.00     24000
 
 [[7985    0    0]
 [   0 8009    0]
 [   0    0 8006]] 

fit_time 788.5838 3.3086
score_time 1.7132 0.0500
test_accuracy 1.0000 0.0000
test_f1_macro 1.0000 0.0000
test_f1_weighted 1.0000 0.0000
test_precision_macro 1.0000 0.0000
test_precision_micro 1.0000 0.0000
test_precision_weighted 1.0000 0.0000
test_recall_macro 1.0000 0.0000
test_recall_micro 1.0000 0.0000
test_recall_weighted 1.0000 0.0000
train_accuracy 1.0000 0.0000
train_f1_macro 1.0000 0.0000
train_f1_weighted 1.0000 0.0000
train_precision_macro 1.0000 0.0000
train_precision_micro 1.0000 0.0000
train_precision_weighted 1.0000 0.0000
train_recall_ma